## Autor: Gabriel Frank Simonetto

#### Email: gabrielfranksimonetto@gmail.com
#### LinkedIn: https://www.linkedin.com/in/gabriel-simonetto/

# Introdução

O texto a seguir é uma análise em campanhas do Facebook Ads com dados anonimizados.

O objetivo principal é fazer com que o máximo de pessoas vejam o anúncio, mas ao mesmo tempo, é importante para uma empresa garantir que está atingindo as pessoas certas em seu plano de marketing, portanto, boa parte da minha análise será também voltada para otimização de conversão e rentabilidade.

# Importando tabela

In [7]:
import pandas as pd    

In [8]:
campanhas = pd.read_excel("Desafio Técnico - Planilha.xlsx")

In [9]:
campanhas.head(5)

,Nome do conjunto de anúncio,Veiculação,Orçamento,Edições significativa mais recente,Alcance,Impressão,Custo por resultados,Iní­cio,Término,Frequência de visualização,Cliques no link únicos,Cliques no link,Porcentagem assistida do vídeo,Tempo médio assistido do vídeo
0,Anúncio 1,active,Usando apenas orçamento da campanha,2018-09-21T16:07:01-0300,26,66,0.28,9/21/18,10/22/18,0.250,1,2,21.51,00:00:03
1,Anúncio 2,active,Usando apenas orçamento da campanha,NaN,55,92,0.21,9/21/18,10/22/18,0.160,2,2,24.19,00:00:03
2,Anúncio 3,active,Usando apenas orçamento da campanha,2018-09-21T15:34:45-0300,6602,10438,0.15,9/21/18,10/22/18,0.150,561,607,47.73,00:00:07
3,Anúncio 4,active,Usando apenas orçamento da campanha,NaN,12,21,0.33,9/21/18,10/22/18,0.175,1,1,21.41,00:00:10
4,Anúncio 5,active,Usando apenas orçamento da campanha,NaN,58,96,0.17,9/21/18,10/22/18,0.160,2,2,32.72,00:00:04


## Tirando lixo e organizando a tabela

In [10]:
# Reconstruindo Id
index = [x for x in range(1,campanhas.shape[0]+1)]
campanhas.index = index
campanhas.columns.name = "Id"; 

#Retirando variaveis que nao serao utilizadas na análise
campanhas.drop('Nome do conjunto de anúncio', axis = 1, inplace = True)
campanhas.drop('Veiculação', axis = 1, inplace = True)
campanhas.drop('Orçamento', axis = 1, inplace = True)
campanhas.drop('Cliques no link', axis = 1, inplace = True)

#Diminuindo o nome
campanhas.rename(columns={'Edições significativa mais recente': 'Ult. edição'}, inplace=True)
#Reajustando nome da tabela início
campanhas.rename(columns={'Iní­cio': 'Início'}, inplace=True)

In [11]:
#Melhorando a visualização de valores
#Não é tão genérico ou elegante quanto poderia ser, mas faz o serviço
campanhas = campanhas.fillna({'Ult. edição': "-------"})

for i in range(1, campanhas.shape[0]+1):
    if (campanhas.loc[i,'Ult. edição'] != "-------"):
        target = campanhas.at[i,'Ult. edição']
        target = target[:10]
        target = target.split("-")
        target = "{}/{}/{}".format(target[1], target[2], target[0][2:4])
        campanhas.loc[i,'Ult. edição'] = target

#Alguns dados estão escritos por extenso na data de inicio
for i in range(1, campanhas.shape[0]+1):
    if (len(str(campanhas.at[i,'Início'])) > 11):
        target = str(campanhas.at[i,'Início'])
        target = target[:10]
        target = target.split("-")
        target = "{}/{}/{}".format(target[1], target[2], target[0][2:4])
        campanhas.at[i, 'Início'] = target    
    
#Mostra prévia da tabela
campanhas.head(5)

Id,Ult. edição,Alcance,Impressão,Custo por resultados,Início,Término,Frequência de visualização,Cliques no link únicos,Porcentagem assistida do vídeo,Tempo médio assistido do vídeo
1,09/21/18,26,66,0.28,9/21/18,10/22/18,0.250,1,21.51,00:00:03
2,-------,55,92,0.21,9/21/18,10/22/18,0.160,2,24.19,00:00:03
3,09/21/18,6602,10438,0.15,9/21/18,10/22/18,0.150,561,47.73,00:00:07
4,-------,12,21,0.33,9/21/18,10/22/18,0.175,1,21.41,00:00:10
5,-------,58,96,0.17,9/21/18,10/22/18,0.160,2,32.72,00:00:04


#### As métricas de tempo não nos são uteis agora, pois já conferimos que as campanhas rodaram tempo o suficiente para serem analisadas. Serão retiradas da tabela por agora para facilitar a análise, mas devem voltar mais tarde quando estivermos devolvendo a tabela para o banco de dados.

### Retirando métricas de tempo

In [12]:
inicio = campanhas['Início']
ult_edicao = campanhas['Ult. edição']
termino = campanhas['Término']

campanhas.drop('Início', axis = 1, inplace = True)
campanhas.drop('Ult. edição', axis = 1, inplace = True)
campanhas.drop('Término', axis = 1, inplace = True)

## Criação de métrica de CTR

In [15]:
campanhas['CTR'] = round((campanhas['Cliques no link únicos'] * 100) /campanhas['Alcance'],2)

### Reorganizando variaveis na planilha

In [16]:
campanhas = campanhas[
['Alcance',
 'Impressão',
 'Cliques no link únicos',
 'CTR',
 'Custo por resultados',
 'Frequência de visualização',
 'Porcentagem assistida do vídeo',
 'Tempo médio assistido do vídeo']
]

## Formatação finalizada

In [17]:
# Mostra prévia da tabela
campanhas.head(5)

Id,Alcance,Impressão,Cliques no link únicos,CTR,Custo por resultados,Frequência de visualização,Porcentagem assistida do vídeo,Tempo médio assistido do vídeo
1,26,66,1,3.85,0.28,0.250,21.51,00:00:03
2,55,92,2,3.64,0.21,0.160,24.19,00:00:03
3,6602,10438,561,8.50,0.15,0.150,47.73,00:00:07
4,12,21,1,8.33,0.33,0.175,21.41,00:00:10
5,58,96,2,3.45,0.17,0.160,32.72,00:00:04


# Analisando as campanhas

## Elencando Alcance

### Mostrar campanhas com Alcance maior que 4000

In [18]:
selecao = campanhas['Alcance'] > 4000
campanhas[selecao]

Id,Alcance,Impressão,Cliques no link únicos,CTR,Custo por resultados,Frequência de visualização,Porcentagem assistida do vídeo,Tempo médio assistido do vídeo
3,6602,10438,561,8.50,0.15,0.150,47.73,00:00:07
6,13304,21419,925,6.95,0.18,0.180,42.73,00:00:05
12,6602,10438,561,8.50,0.15,0.150,47.73,00:00:07
15,13304,21419,925,6.95,0.18,0.180,42.73,00:00:08
18,6602,10438,561,8.50,0.15,0.150,47.73,00:00:07
19,5003,8693,450,8.99,0.13,0.175,21.41,00:00:10
21,10304,18419,925,8.98,0.09,0.200,42.73,00:00:08
27,6602,10438,561,8.50,0.15,0.130,47.73,00:00:07


### Mostrar campanhas com Alcance entre 100 e 4000

In [19]:
selecao = (campanhas['Alcance'] <= 4000) & (campanhas['Alcance'] > 100)
campanhas[selecao]

Id,Alcance,Impressão,Cliques no link únicos,CTR,Custo por resultados,Frequência de visualização,Porcentagem assistida do vídeo,Tempo médio assistido do vídeo
9,437,569,30,6.86,0.13,0.13,59.68,00:00:08
10,553,659,35,6.33,0.15,0.11,67.39,00:00:09
11,413,524,31,7.51,0.15,0.12,49.57,00:00:06
24,437,569,30,6.86,0.13,0.13,59.68,00:00:08
25,553,659,35,6.33,0.15,0.10,67.39,00:00:09
26,413,524,31,7.51,0.15,0.12,49.57,00:00:06
29,530,860,200,37.74,0.30,0.20,32.72,00:00:03
30,509,872,320,62.87,0.25,0.13,42.73,00:00:08


### Mostrar campanhas com Alcance menor que 100

In [20]:
selecao = (campanhas['Alcance'] <= 100)
campanhas[selecao]

Id,Alcance,Impressão,Cliques no link únicos,CTR,Custo por resultados,Frequência de visualização,Porcentagem assistida do vídeo,Tempo médio assistido do vídeo
1,26,66,1,3.85,0.28,0.250,21.51,00:00:03
2,55,92,2,3.64,0.21,0.160,24.19,00:00:03
4,12,21,1,8.33,0.33,0.175,21.41,00:00:10
5,58,96,2,3.45,0.17,0.160,32.72,00:00:04
7,2,6,0,0.00,2.60,0.010,10.15,00:00:00
8,2,9,0,0.00,2.10,0.020,12.15,00:00:00
13,50,100,10,20.00,0.29,0.175,21.41,00:00:10
14,59,97,2,3.39,0.19,0.160,32.72,00:00:03
16,50,70,10,20.00,1.01,0.250,21.51,00:00:03
17,55,92,2,3.64,0.21,0.160,24.19,00:00:03


### Mostrar campanhas com CPR maior que 1.00

In [21]:
selecao = campanhas['Custo por resultados'] > 1.00
campanhas[selecao]

Id,Alcance,Impressão,Cliques no link únicos,CTR,Custo por resultados,Frequência de visualização,Porcentagem assistida do vídeo,Tempo médio assistido do vídeo
7,2,6,0,0.00,2.60,0.01,10.15,00:00:00
8,2,9,0,0.00,2.10,0.02,12.15,00:00:00
16,50,70,10,20.00,1.01,0.25,21.51,00:00:03
22,63,82,13,20.63,2.60,0.01,10.15,00:00:05
23,25,46,6,24.00,6.10,0.02,12.15,00:00:09


### Mostrar campanhas com CTR bom, mas Alcance fraco

In [22]:
selecao = (campanhas['Alcance'] < 4000) & (campanhas['CTR'] > 10.0)
campanhas[selecao]

Id,Alcance,Impressão,Cliques no link únicos,CTR,Custo por resultados,Frequência de visualização,Porcentagem assistida do vídeo,Tempo médio assistido do vídeo
13,50,100,10,20.00,0.29,0.175,21.41,00:00:10
16,50,70,10,20.00,1.01,0.250,21.51,00:00:03
22,63,82,13,20.63,2.60,0.010,10.15,00:00:05
23,25,46,6,24.00,6.10,0.020,12.15,00:00:09
28,50,100,10,20.00,0.29,0.175,21.41,00:00:10
29,530,860,200,37.74,0.30,0.200,32.72,00:00:03
30,509,872,320,62.87,0.25,0.130,42.73,00:00:08


### Campanhas com Alcance bom, mas CTR fraco

In [24]:
selecao = (campanhas['Alcance'] > 4000) & (campanhas['CTR'] < 10.0)
campanhas[selecao]

Id,Alcance,Impressão,Cliques no link únicos,CTR,Custo por resultados,Frequência de visualização,Porcentagem assistida do vídeo,Tempo médio assistido do vídeo
3,6602,10438,561,8.50,0.15,0.150,47.73,00:00:07
6,13304,21419,925,6.95,0.18,0.180,42.73,00:00:05
12,6602,10438,561,8.50,0.15,0.150,47.73,00:00:07
15,13304,21419,925,6.95,0.18,0.180,42.73,00:00:08
18,6602,10438,561,8.50,0.15,0.150,47.73,00:00:07
19,5003,8693,450,8.99,0.13,0.175,21.41,00:00:10
21,10304,18419,925,8.98,0.09,0.200,42.73,00:00:08
27,6602,10438,561,8.50,0.15,0.130,47.73,00:00:07


### Campanhas com Alcance e CTR fracos

In [25]:
selecao = (campanhas['Alcance'] < 4000) & (campanhas['CTR'] < 10.0)
campanhas[selecao]

Id,Alcance,Impressão,Cliques no link únicos,CTR,Custo por resultados,Frequência de visualização,Porcentagem assistida do vídeo,Tempo médio assistido do vídeo
1,26,66,1,3.85,0.28,0.250,21.51,00:00:03
2,55,92,2,3.64,0.21,0.160,24.19,00:00:03
4,12,21,1,8.33,0.33,0.175,21.41,00:00:10
5,58,96,2,3.45,0.17,0.160,32.72,00:00:04
7,2,6,0,0.00,2.60,0.010,10.15,00:00:00
8,2,9,0,0.00,2.10,0.020,12.15,00:00:00
9,437,569,30,6.86,0.13,0.130,59.68,00:00:08
10,553,659,35,6.33,0.15,0.110,67.39,00:00:09
11,413,524,31,7.51,0.15,0.120,49.57,00:00:06
14,59,97,2,3.39,0.19,0.160,32.72,00:00:03


### Testes de hipótese de correlação

In [26]:
#Entre Frequência de Visualização e CTR
campanhas['Frequência de visualização'].corr(campanhas['CTR'])

0.04382119161704458

In [27]:
#Entre Frequência de Visualização e Alcance
campanhas['Frequência de visualização'].corr(campanhas['Alcance'])

0.2571568269192032

In [29]:
#Entre Frequência de Visualização e Cliques no link únicos
campanhas['Frequência de visualização'].corr(campanhas['Cliques no link únicos'])

0.2732538748471476

## Exportando tabela para uso posterior

In [ ]:
campanhas.to_excel('tabelaCurada.xlsx')